In [17]:
!pip install spacy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from src.utils import load_all_restaurants
import pandas as pd
import spacy
from nltk.tokenize import word_tokenize

In [63]:
df = load_all_restaurants("data\\raw")
df.head()

,title,description,n_reviews,stars_avg,price_range,extra_features,street,opening_hours,phone_number,reviews,extra_content,unique_id
0,Restaurante Abando,Restaurante de larga trayectoria con pinchos c...,3000,"4,3",€€,Tiene terraza· Tiene Wi-Fi· Sillas altas dispo...,"Colón de Larreátegui K., 31, Abando, 48009 Bil...","{'viernes': ['7:00–1:30'], 'sábado': ['7:00–1:...",944 24 57 79,['Agradable lugar para comer tanto a la carta ...,['Abando Bar Restaurante. Colón de Larreátegui...,"Restaurante Abando | Colón de Larreátegui K., ..."
1,"Bar-Restaurante ""La Roca""",NaN,11000,"4,2",20-30€,Ideal para ver deportes· Sillas altas disponib...,"Archivo Histórico Provincial, Ercilla Kalea, 1...","{'sábado': ['12:00–24:00'], 'domingo': ['Cerra...",944 25 50 09,"['No suelo poner reseñas, pero esta vez, lo me...",['... café baristas cerveza metodología de tir...,"Bar-Restaurante ""La Roca"" | Archivo Histórico ..."
2,La Gavilla Bilbao,Restaurante funcional y acogedor que ofrece pl...,11000,"4,5",30-40€,NaN,"Colón de Larreátegui K., N.32, Abando, 48009 B...","{'sábado': ['12:00–24:00'], 'domingo': ['Cerra...",944 25 68 38,"['Veamos, sólo comí cuatro pinchos que estaban...",['Céntrico establecimiento que se ha puesto ba...,"La Gavilla Bilbao | Colón de Larreátegui K., N..."
3,La Kedada Bilbao,Bar restaurante de un hotel moderno y de pared...,54,"4,0",NaN,Tiene terraza· Tiene comedor privado· Tiene Wi-Fi,"Rodríguez Arias K., 66, Bajo, Abando, 48013 Bi...","{'sábado': ['19:00–23:00'], 'domingo': ['19:00...",944 41 31 00,['La comida esta rica lo que pasa que algunos ...,['El restaurante Bar Restaurante La Kedada est...,"La Kedada Bilbao | Rodríguez Arias K., 66, Baj..."
4,Restaurante Aralar,NaN,883,"4,3",10-20€,Se puede bailar· Tiene Wi-Fi· Sillas altas dis...,"Ibáñez de Bilbao Kalea, nº 16, Abando, 48001 B...","{'sábado': ['12:00–21:00'], 'domingo': ['Cerra...",944 23 09 52,['No me gusta mucho ir a sitios de ningún part...,['Calle Ibáñez de Bilbao 16 Sabin Etxea. 48001...,"Restaurante Aralar | Ibáñez de Bilbao Kalea, n..."


In [64]:
df = df.drop(columns=['unique_id'])
df = df.drop(columns=['description'])
df = df.drop(columns=['stars_avg'])
df = df.drop(columns=['opening_hours'])
df = df.drop(columns=['phone_number'])
df = df.drop(columns=['extra_content'])
df = df.drop(columns=['reviews'])
df.head()

,title,n_reviews,price_range,extra_features,street
0,Restaurante Abando,3000,€€,Tiene terraza· Tiene Wi-Fi· Sillas altas dispo...,"Colón de Larreátegui K., 31, Abando, 48009 Bil..."
1,"Bar-Restaurante ""La Roca""",11000,20-30€,Ideal para ver deportes· Sillas altas disponib...,"Archivo Histórico Provincial, Ercilla Kalea, 1..."
2,La Gavilla Bilbao,11000,30-40€,NaN,"Colón de Larreátegui K., N.32, Abando, 48009 B..."
3,La Kedada Bilbao,54,NaN,Tiene terraza· Tiene comedor privado· Tiene Wi-Fi,"Rodríguez Arias K., 66, Bajo, Abando, 48013 Bi..."
4,Restaurante Aralar,883,10-20€,Se puede bailar· Tiene Wi-Fi· Sillas altas dis...,"Ibáñez de Bilbao Kalea, nº 16, Abando, 48001 B..."


In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\USUARIO.DESKTOP-
[nltk_data]     GN89KO5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
# Reemplazar NaN por cadena vacía
df.fillna("", inplace=True)

# Separar las categorías y limpiar los espacios en blanco
df['extra_features'] = df['extra_features'].str.split('·').apply(lambda x: '_'.join([f.strip() for f in x]))

# Convertir las características en variables ficticias (dummies)
dummies = df['extra_features'].str.get_dummies(sep='_')

# Agregar las variables ficticias al DataFrame original
df = pd.concat([df, dummies], axis=1)

# Eliminar la columna original "extra_features"
df.drop(['extra_features'], axis=1, inplace=True)

df.head()


,title,n_reviews,price_range,street,Es obligatorio hacer una reserva,Happy hour de comida,Hay actuaciones en directo,Hay chimenea,Hay karaoke,Hay menú infantil,...,Sirve cenas,Solo se acepta efectivo,Tiene Wi-Fi,Tiene azotea,Tiene cambiador,Tiene comedor privado,Tiene happy hour,Tiene juegos de mesa,Tiene música en directo,Tiene terraza
0,Restaurante Abando,3000,€€,"Colón de Larreátegui K., 31, Abando, 48009 Bil...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,"Bar-Restaurante ""La Roca""",11000,20-30€,"Archivo Histórico Provincial, Ercilla Kalea, 1...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,La Gavilla Bilbao,11000,30-40€,"Colón de Larreátegui K., N.32, Abando, 48009 B...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,La Kedada Bilbao,54,,"Rodríguez Arias K., 66, Bajo, Abando, 48013 Bi...",0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,Restaurante Aralar,883,10-20€,"Ibáñez de Bilbao Kalea, nº 16, Abando, 48001 B...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [66]:
df['price_range'] = df['price_range'].replace({
    'Más de 100€': '+100€',
    '€': '10-40€',
    '€€': '40-80€',
    '€€€': '+100€'
})

#Función para asignar la categoría.
def categorize_price(price_range):
    # Verificar si el valor es un string
    if isinstance(price_range, str):
        # Dividir el rango de precios y obtener el último fragmento
        last_fragment = price_range.split('-')[-1]
        
        # Buscar números en el último fragmento
        numbers = re.findall(r'\d+', last_fragment)
        
        # Verificar si hay números encontrados
        if numbers:
            # Obtener el precio máximo del rango
            price_max = int(numbers[0])
            
            # Asignar categoría
            if price_max <= 10:
                return 'barato'
            elif price_max <= 20:
                return 'medio'
            else:
                return 'caro'
        else:
            # Manejar el caso donde no se encuentra ningún número
            return 'No válido'
    else:
        # Manejar el caso donde el valor no es un string
        return 'No válido'
    
    
df['price_range'] = df['price_range'].apply(categorize_price)

In [67]:
# Función para clasificar las reseñas
def classify_reviews(n_reviews):
    if n_reviews < 100:
        return "pocas_resenas"
    elif 100 <= n_reviews <= 800:
        return "medio_resenas"
    else:
        return "muchas_resenas"

# Aplicar la clasificación a la columna 'n_reviews'
df['n_reviews'] = df['n_reviews'].apply(classify_reviews)
# Convertir la columna 'review_category' a categórica
df['n_reviews'] = pd.Categorical(df['n_reviews'], categories=["pocas_resenas", "medio_resenas", "muchas_resenas"], ordered=True)

In [68]:
df

,title,n_reviews,price_range,street,Es obligatorio hacer una reserva,Happy hour de comida,Hay actuaciones en directo,Hay chimenea,Hay karaoke,Hay menú infantil,...,Sirve cenas,Solo se acepta efectivo,Tiene Wi-Fi,Tiene azotea,Tiene cambiador,Tiene comedor privado,Tiene happy hour,Tiene juegos de mesa,Tiene música en directo,Tiene terraza
0,Restaurante Abando,muchas_resenas,caro,"Colón de Larreátegui K., 31, Abando, 48009 Bil...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,"Bar-Restaurante ""La Roca""",muchas_resenas,caro,"Archivo Histórico Provincial, Ercilla Kalea, 1...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,La Gavilla Bilbao,muchas_resenas,caro,"Colón de Larreátegui K., N.32, Abando, 48009 B...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,La Kedada Bilbao,pocas_resenas,No válido,"Rodríguez Arias K., 66, Bajo, Abando, 48013 Bi...",0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,Restaurante Aralar,muchas_resenas,medio,"Ibáñez de Bilbao Kalea, nº 16, Abando, 48001 B...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Gaztelu bar-degustación,pocas_resenas,caro,"Iturribide Zeharkalea, 4, Ibaiondo, 48006 Bilb...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93,Bar Amaia,pocas_resenas,caro,"Santutxu Kalea, 71, 48006 Bilbo, Bizkaia",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94,Cocherito,pocas_resenas,barato,"Cocherito de Bilbao Kalea, 1, 48004 Bilbo, Biz...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Xukela,muchas_resenas,medio,"Txakur Kalea, 2, Ibaiondo, 48005 Bilbo, Bizkaia",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
